# <center>**Encoding data in BUFR PART 1** </center>

The goal of this notebook is to show how to encode a simple csv file with buoy wave data
To do so we import the eccodes library and others ( pandas, datetime)

In [27]:
import eccodes as ecc
import pandas as pd
from datetime import datetime

Now we have to read the csv into a pandas dataframe

In [28]:
obsFile="/home/marg/ECCODES_2025/ecTrain/nassau.csv"
df=pd.read_csv(obsFile)
print(df.head())

      id       ymd  hhmm  latitude  longitude  wheight  avg_period  pressure
0  41047  20250929  1810     27.56     -71.48      4.1         9.9    1009.4


once we read the observations file we can create a function that encodes the dataframe into BUFR

In [29]:
def encode(df_obs,outputFile):
    bid=ecc.codes_bufr_new_from_samples('BUFR4')
    # here we create the handle to a bufr message using BUFR4 as a 'template'
    ecc.codes_set(bid,'masterTablesVersionNumber',42)
    ecc.codes_set(bid,'localTablesVersionNumber',0)
    ecc.codes_set(bid,'compressedData',1)
    ecc.codes_set(bid,'numberOfSubsets',1)
    # here, we read the YMD and HHMM to build the time stamp
    # this timestamp is used to populate the BUFR time keys
    ymd=df_obs['ymd'].values[0]
    hhmm=df_obs['hhmm'].values[0] 
    str_ymdhm=str(ymd)+str(hhmm)
    timeStamp=datetime.strptime(str_ymdhm,'%Y%m%d%H%M')
    ecc.codes_set(bid,"typicalYear",timeStamp.year)
    ecc.codes_set(bid,"typicalMonth",timeStamp.month)
    ecc.codes_set(bid,"typicalDay",timeStamp.day)
    ecc.codes_set(bid,"typicalHour",timeStamp.hour)
    ecc.codes_set(bid,"typicalMinute",timeStamp.minute)
    # here we define the BUFR structure through the descriptors we want to use
    unexpandedDescr=[ 1015,4001,4002,4003,4004,4005,
                     5001,6001,22021,22011,10051]
    ecc.codes_set_array(bid,'unexpandedDescriptors',unexpandedDescr)
    stationId=str(df['id'].values[0])
    ecc.codes_set(bid,'stationOrSiteName',stationId)
    # populate the data section
    ecc.codes_set(bid,"year",timeStamp.year)
    ecc.codes_set(bid,"month",timeStamp.month)
    ecc.codes_set(bid,"day",timeStamp.day)
    ecc.codes_set(bid,"hour",timeStamp.hour)
    ecc.codes_set(bid,"minute",timeStamp.minute)
    lat=df_obs["latitude"].values
    lon=df_obs["longitude"].values 
    height=df_obs['wheight'].values 
    period=df_obs['avg_period'].values 
    pressure=df_obs['pressure'].values
    ecc.codes_set_array(bid,"latitude",lat)
    ecc.codes_set_array(bid,"longitude",lon)
    ecc.codes_set_array(bid,"heightOfWaves",height)
    ecc.codes_set_array(bid,"periodOfWaves",period)
    ecc.codes_set_array(bid,"pressureReducedToMeanSeaLevel",pressure)
    ecc.codes_set(bid,'pack',1) 
    # here we pack the data and the descriptors "so to speak" so they can be written to the output file
    with open(outputFile,'wb') as fout:
        ecc.codes_write(bid,fout) # we use codes_write to copy the handle bid into the file obj fout
    ecc.codes_release(bid)
    print(f"creating output bufr file : {outputFile}")
    return

In [30]:
def main():
    obsFile="/home/marg/ECCODES_2025/ecTrain/nassau.csv"
    df=pd.read_csv(obsFile)
    outputBufrFile="/home/marg/ECCODES_2025/ecTrain/nassau1.b"
    encode(df,outputBufrFile)
if __name__=="__main__":
    main()

creating output bufr file : /home/marg/ECCODES_2025/ecTrain/nassau1.b


In [26]:
def main():
    obsFile="/home/marg/ECCODES_2025/ecTrain/nassau.csv"
    df=pd.read_csv(obsFile)
    outputBufrFile="/home/marg/ECCODES_2025/ecTrain/nassau1.b"
    encode(df,outputBufrFile)
if __name__=="__main__":
    main()

creating output bufr file : /home/marg/ECCODES_2025/ecTrain/nassau1.b
